# Análise comparativa de resultados


## 1. Preparação dos dados

### 1.1 Configurações iniciais

In [1]:
import pandas as pd
import seaborn as sns

from pathlib import Path

%pip install -U scikit-learn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import (
    OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, LabelEncoder
)


Note: you may need to restart the kernel to use updated packages.


### 1.2 Obtendo os dados

In [12]:
data_path = Path('../data/raw/data.csv')
dict_path = Path('../data/external/dicionario.csv')

In [13]:
# Ler conjunto de dados
df = (
    pd.read_csv(data_path)
)
# Ler dicionário de dados
df_dict = pd.read_csv(dict_path)
df_dict

,variavel,significado,tipo,subtipo,resposta
0,gender,Gênero do aluno,Qualitativa,Nominal,False
1,NationalITy,Nacionalidade do aluno,Qualitativa,Nominal,False
2,PlaceofBirth,Local de nascimento do aluno,Qualitativa,Nominal,False
3,StageID,Nível de escolaridade a que o aluno pertence,Qualitativa,Ordinal,False
4,GradeID,A série em que o aluno está matriculado,Qualitativa,Ordinal,False
5,SectionID,Sala de aula à qual o aluno pertence,Qualitativa,Nominal,False
6,Topic,Tópico do curso,Qualitativa,Nominal,False
7,Semester,Semestre do ano letivo,Qualitativa,Ordinal,False
8,Relation,Progenitor responsável pelo aluno,Qualitativa,Nominal,False
9,raisedhands,Número de vezes que o aluno levantou a mão,Quantitativa,Discreta,False


In [14]:
# Separar variáveis

variaveis_interesse = df_dict.query('resposta == False').variavel.to_list()
variavel_resposta = df_dict.query('resposta == True').variavel.to_list()


df_X = df[variaveis_interesse]
df_y = df[variavel_resposta]

# Separar variáveis de interesse por tipo
nominal_columns = df_dict.query('variavel in @variaveis_interesse and subtipo == "Nominal"').variavel.to_list()
continuos_columns = df_dict.query('variavel in @variaveis_interesse and subtipo == "Discreta"').variavel.to_list()

### 1.3 Tratando dados discrepantes

In [15]:
# criando pipelines para tratar os dados

nominal_preprocessor = Pipeline(steps=[
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes ()
    ('encoding', OneHotEncoder(sparse=False, handle_unknown='ignore')), # Codificação de variáveis
    # Seleção de variáveis
     ('normalization', StandardScaler()) # Normalização de variáveis
])
continuous_preprocessor = Pipeline(steps=[
    # Tratamento de dados discrepantes
    ('missing', KNNImputer(n_neighbors=5)), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', RobustScaler()) # Normalização
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuos', continuous_preprocessor, continuos_columns)
])

preprocessor

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False)),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['gender', 'NationalITy', 'PlaceofBirth',
                                  'SectionID', 'Topic', 'Relation',
                                  'ParentAnsweringSurvey',
                                  'ParentschoolSatisfaction']),
                                ('continuos',
                                 Pipeline(steps=[('missing', KNNImputer()),
                                                 ('normalization',
                                                  RobustScaler())]),
                                 ['raisedhands', 'VisITedResources',
                                  'AnnouncementsView', 'Discussion'])])

In [16]:
preprocessor.fit(df_X)

X_transformed = preprocessor.transform(df_X)

X_transformed

C:\Users\letic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[-0.7574764 ,  0.7574764 , -0.1382327 , ..., -0.765625  ,
        -0.70454545, -0.38      ],
       [-0.7574764 ,  0.7574764 , -0.1382327 , ..., -0.703125  ,
        -0.68181818, -0.28      ],
       [-0.7574764 ,  0.7574764 , -0.1382327 , ..., -0.90625   ,
        -0.75      , -0.18      ],
       ...,
       [ 1.32017315, -1.32017315, -0.1382327 , ...,  0.140625  ,
        -0.18181818, -0.2       ],
       [ 1.32017315, -1.32017315, -0.1382327 , ..., -0.75      ,
        -0.43181818,  0.36      ],
       [ 1.32017315, -1.32017315, -0.1382327 , ..., -0.796875  ,
        -0.22727273,  0.46      ]])